In [ ]:
#labelencoder, colsample_bytree=0.8
#private Score: 0.86488, Public score: 0.87923
#增加total_contact、contact_diff
#Private score: 0.86355，Public score: 0.87676
#保留RecordID
#Private score: 0.85822，Public score: 0.86590
        
#targetencoder
#private Score: 0.86577, Public score: 0.87771
#增加total_contact、contact_diff
#Private score: 0.86622, Public score: 0.87790
#保留RecordID
#Private score: 0.86888，Public score: 0.88209
#使用全部数据训练，不做train_test_split
#*Private score: 0.87111，Public score: 0.88209

#onehot + XGBoost
#Private score: 0.86666, Public score: 0.87847
#tree_method = 'hist'
#Private score: 0.86400, Public score: 0.87904
#tree_method = 'hist', subsample=1
#Private score: 0.86666, Public score: 0.87771
#增加total_contact、contact_diff
#Private score: 0.86711, Public score: 0.87828
#增加total_contact、contact_diff, subsample=1, 不使用tree_method = 'hist'
#Private score: 0.86577, Public score: 0.87790
#改回subsample=0.8, 
#Private score: 0.86844, Public score: 0.87733
#改回subsample=0.8,tree_method = 'hist'
#Private score: 0.86622, Public score: 0.87676
#保留RecordID
#Private score: 0.86800，Public score: 0.88514
#使用全部数据训练，不做train_test_split
#Private score: 0.86977，Public score: 0.88228


In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib # 注意这个也要import一次
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, mean_squared_error, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#加载数据
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [ ]:
datas = pd.concat([train_data, test_data], axis=0)

In [ ]:
datas.columns

In [ ]:
cat_columns = datas.select_dtypes(include='O').columns

In [ ]:
cat_columns

In [ ]:
#观察分类特征与target的关联，关注一些重要特征
sns.countplot(x='poutcome', hue = 'subscribe', data=train_data)
#检查数据集是否平衡
# sns.countplot(x='subscribe', data=train_data)

### label encoder

In [ ]:
datas['job'] = datas['job'].map({'unknown': 0, 'admin.': 1, 'services': 2, 'blue-collar': 3, 'entrepreneur': 4, 'management': 5,
                  'technician': 6, 'housemaid': 7, 'self-employed': 8, 'unemployed': 9, 'retired': 10, 'student': 11})

In [ ]:
datas['marital'] = datas['marital'].map({'unknown': 0, 'divorced': 1, 'married': 2, 'single': 3})

In [ ]:
datas['education'] = datas['education'].map({'unknown': 0, 'illiterate': 1, 'basic.4y': 2, 'basic.6y': 3, 'basic.9y': 4,
                                             'high.school': 5, 'university.degree': 6, 'professional.course': 7})

In [ ]:
datas['default'] = datas['default'].map({'unknown': 0, 'no': 1, 'yes': 2})

In [ ]:
datas['housing'] = datas['housing'].map({'unknown': 0, 'no': 1, 'yes': 2})

In [ ]:
datas['loan'] = datas['loan'].map({'unknown': 0, 'no': 1, 'yes': 2})

In [ ]:
datas['contact'] = datas['contact'].map({'cellular': 0, 'telephone': 1})

In [ ]:
datas['month'] = datas['month'].map({'mar':3, 'apr': 4, 'may': 5, 'jun': 6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov': 11, 'dec': 12})

In [ ]:
datas['day_of_week'] = datas['day_of_week'].map({'mon': 0, 'tue': 1, 'wed': 2, 'thu': 3, 'fri': 4})

In [ ]:
datas['poutcome'] = datas['poutcome'].map({'nonexistent': 0, 'failure': 1, 'success': 2})

In [ ]:
datas['subscribe'] = datas['subscribe'].map({'no': 0, 'yes': 1})

In [ ]:
#去掉ID
# datas.drop(['RecordID'], axis=1, inplace=True)

## 特征工程

In [ ]:
datas['total_contact'] = datas['campaign'] + datas['previous']
datas['contact_diff'] = datas['campaign'] - datas['previous']

In [ ]:
# 数据集切分
train = datas[datas['subscribe'].notnull()]
test = datas[datas['subscribe'].isnull()]

In [ ]:
train_X = train.drop(['subscribe'], axis=1)
train_y = train['subscribe']
test_X = test.drop(['subscribe'], axis=1)

In [ ]:
#增加检验
train_X,eval_X,train_y,eval_y = train_test_split(train_X, train_y, test_size=0.2, random_state=2022, stratify=train_y)

## 使用lgbm 训练

In [ ]:
import lightgbm as lgb

In [ ]:
model_lgb = lgb.LGBMClassifier(
            num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='binary', metric = 'binary_logloss,auc',
            max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2022,
            n_estimators=2000, subsample=1, colsample_bytree=0.8,
        )

In [ ]:
model_lgb.fit(train_X, train_y, eval_set=(eval_X, eval_y), #categorical_feature = cat_columns.values.tolist(),
              early_stopping_rounds = 200)

# model_lgb.fit(train_X, train_y)

In [ ]:
fig, ax = plt.subplots(figsize=(6,18))
lgb.plot_importance(model_lgb, max_num_features=70, ax = ax)

In [ ]:
y_pred = model_lgb.predict(test_X )#[:, 1]

In [ ]:
subscribe_map = {0: 'no', 1: 'yes'}

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')
submission['subscribe'] = [subscribe_map[x] for x in y_pred]
submission.to_csv('./submission/submission.csv', index=None)

## targetencoser

In [ ]:
train2 = train_data.copy()
test2 = test_data.copy()

In [ ]:
train2['subscribe'] = train2['subscribe'].map({'no': 0, 'yes': 1})

In [ ]:
cat_columns = test_data.select_dtypes(include='O').columns

In [ ]:
for col in cat_columns:
    temp = train2.groupby(col)['subscribe'].agg(['mean', 'std', 'count']).reset_index()
    temp.columns = [col, col + '_mean', col + '_std', col + '_count']
    train2 = train2.merge(temp, on=col, how= 'left')
    test2 = test2.merge(temp, on=col, how= 'left')

In [ ]:
# train2.drop(columns=['RecordID'], inplace=True)
train2.drop(columns=cat_columns, inplace=True)

# test2.drop(columns=['RecordID'], inplace=True)
test2.drop(columns=cat_columns, inplace=True)

In [ ]:
train2['total_contact'] = train2['campaign'] + train2['previous']
train2['contact_diff'] = train2['campaign'] - train2['previous']

test2['total_contact'] = test2['campaign'] + test2['previous']
test2['contact_diff'] = test2['campaign'] - test2['previous']

In [ ]:
train2_X = train2.drop(['subscribe'], axis=1)
train2_y = train2['subscribe']
test2_X = test2

In [ ]:
#增加检验
train2_X,eval2_X,train2_y,eval2_y = train_test_split(train2_X, train2_y, test_size=0.2, random_state=2022, stratify=train2_y)

In [ ]:
model_lgb = lgb.LGBMClassifier(
            num_leaves=2**5-1, reg_alpha=0.25, reg_lambda=0.25, objective='binary', metric = 'binary_logloss,auc',
            max_depth=-1, learning_rate=0.005, min_child_samples=3, random_state=2022,
            n_estimators=2000, subsample=1, colsample_bytree=0.8,
        )

In [ ]:
model_lgb.fit(train2_X, train2_y, eval_set=(eval2_X, eval2_y), #categorical_feature = cat_columns.values.tolist(),
              early_stopping_rounds = 200)

# model_lgb.fit(train2_X, train2_y)

In [ ]:
fig, ax = plt.subplots(figsize=(6,18))
lgb.plot_importance(model_lgb, max_num_features=70, ax = ax)

In [ ]:
y_pred = model_lgb.predict(test2_X )#[:, 1]

In [ ]:
subscribe_map = {0: 'no', 1: 'yes'}
submission = pd.read_csv('./data/sample_submission.csv')
submission['subscribe'] = [subscribe_map[x] for x in y_pred]
submission.to_csv('./submission/submission_targetencoder.csv', index=None)

## XGBoost + onehot encoder

In [ ]:
datas3 = pd.concat([train_data, test_data], axis=0)

In [ ]:
datas3['subscribe'] = datas3['subscribe'].map({'no': 0, 'yes': 1})

In [ ]:
datas3 = pd.get_dummies(datas3)

In [ ]:
datas3

In [ ]:
#去掉ID
# datas3.drop(['RecordID'], axis=1, inplace=True)

In [ ]:
datas3['total_contact'] = datas3['campaign'] + datas3['previous']
datas3['contact_diff'] = datas3['campaign'] - datas3['previous']

In [ ]:
# 数据集切分
train3 = datas3[datas3['subscribe'].notnull()]
test3 = datas3[datas3['subscribe'].isnull()]

In [ ]:
train3_X = train3.drop(['subscribe'], axis=1)
train3_y = train3['subscribe']
test3_X = test3.drop(['subscribe'], axis=1)

In [ ]:
#增加检验
train3_X,eval3_X,train3_y,eval3_y = train_test_split(train3_X, train3_y, test_size=0.2, random_state=2022, stratify=train3_y)

In [ ]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier(
            max_depth=6, learning_rate=0.05, n_estimators=2000, objective='binary:logistic', # tree_method='gpu_hist', 
            subsample=0.8, colsample_bytree=0.8, min_child_samples=3, eval_metric='auc', reg_lambda=0.5
        )

In [ ]:
model_xgb.fit(train3_X, train3_y, eval_set=[(eval3_X, eval3_y)], early_stopping_rounds = 200)

# model_xgb.fit(train3_X, train3_y)

In [ ]:
model_xgb.best_iteration, model_xgb.best_score

In [ ]:
fig, ax = plt.subplots(figsize=(6,18))
xgb.plot_importance(model_xgb, max_num_features=70, ax = ax)

In [ ]:
y_pred = model_xgb.predict(test3_X)#[:, 1]

In [ ]:
subscribe_map = {0: 'no', 1: 'yes'}

submission = pd.read_csv('./data/sample_submission.csv')
submission['subscribe'] = [subscribe_map[x] for x in y_pred]
submission.to_csv('./submission/submission_xgboost.csv', index=None)